In [1]:
import logging
from astropy.table import Table, join

from plato.stars import gaia
from plato.utils import get_abspath

logging.getLogger("astroquery").setLevel(logging.WARNING)

A new version of galpy (1.10.0) is available, please upgrade using pip/conda/... to get the latest features and bug fixes!


In [2]:
# load gaia IDs (DR2) from asPIC catalog
asPIC_ids = Table.read(get_abspath() + "data/external/asPIC_1.1.fits")["sourceId"][
    :, 0
].tolist()

In [3]:
# query the dr2_d3_crossmatch table to get the dr3 ids for the asPIC (dr2) sources
def dr2_dr3_crossmatch_query(id_list):
    query = f"""
    SELECT
        dr2_source_id,
        dr3_source_id,
        angular_distance,
        magnitude_difference
    FROM
        gaiadr3.dr2_neighbourhood
    WHERE
        dr2_source_id IN ({id_list})
  """
    return query


crossmatch_catalogue = gaia.query(dr2_dr3_crossmatch_query, asPIC_ids).to_pandas()

# select the closest match (in terms of angular distance) for each asPIC source
crossmatch_catalogue = crossmatch_catalogue.sort_values(
    by=["dr2_source_id", "angular_distance", "magnitude_difference"]
)
crossmatch_catalogue = crossmatch_catalogue.drop_duplicates(
    subset="dr2_source_id", keep="first"
).rename(columns={"dr3_source_id": "SOURCE_ID"})

# turn dataframe into back into astropy table
crossmatch_catalogue = Table.from_pandas(
    crossmatch_catalogue[["dr2_source_id", "SOURCE_ID"]]
)

100%|██████████| 54/54 [08:23<00:00,  9.32s/it]


In [4]:
# create query to get relevant columns from Gaia DR3 gaia_source and astrophysical_parameters tables
def data_query(id_list):
    query = f"""
    SELECT
        gaiadr3.gaia_source.source_id,
        gaiadr3.gaia_source.parallax,
        gaiadr3.gaia_source.parallax_error,
        gaiadr3.gaia_source.ra,
        gaiadr3.gaia_source.ra_error,
        gaiadr3.gaia_source.dec,
        gaiadr3.gaia_source.dec_error,
        gaiadr3.gaia_source.pmra,
        gaiadr3.gaia_source.pmra_error,
        gaiadr3.gaia_source.pmdec,
        gaiadr3.gaia_source.pmdec_error,
        gaiadr3.gaia_source.radial_velocity,
        gaiadr3.gaia_source.radial_velocity_error,
        astrophysical_parameters.mh_gspphot,
        astrophysical_parameters.mh_gspphot_lower,
        astrophysical_parameters.mh_gspphot_upper,
        astrophysical_parameters.mh_gspspec,
        astrophysical_parameters.mh_gspspec_lower,
        astrophysical_parameters.mh_gspspec_upper,
        astrophysical_parameters.alphafe_gspspec,
        astrophysical_parameters.alphafe_gspspec_lower,
        astrophysical_parameters.alphafe_gspspec_upper,
        astrophysical_parameters.logg_gspphot,
        astrophysical_parameters.logg_gspphot_lower,
        astrophysical_parameters.logg_gspphot_upper,
        astrophysical_parameters.logg_gspspec,
        astrophysical_parameters.logg_gspspec_lower,
        astrophysical_parameters.logg_gspspec_upper,
        astrophysical_parameters.flags_gspspec
    FROM
        gaiadr3.gaia_source
    LEFT JOIN
        gaiadr3.astrophysical_parameters
    ON
        gaiadr3.gaia_source.source_id = gaiadr3.astrophysical_parameters.source_id
    WHERE
        gaiadr3.gaia_source.source_id IN ({id_list});
    """
    return query


targets = gaia.query(data_query, crossmatch_catalogue["SOURCE_ID"].tolist())  # type: ignore

100%|██████████| 54/54 [11:25<00:00, 12.70s/it]


In [5]:
# add DR 2 source IDs back to the table
targets = join(targets, crossmatch_catalogue, keys="SOURCE_ID", join_type="left")

In [6]:
targets["flags_gspspec"] = targets["flags_gspspec"].astype(str)

In [7]:
# save
targets.write(
    get_abspath() + "data/raw/plato_targets.fits", format="fits", overwrite=True
)